<a href="https://colab.research.google.com/github/MehtA-AI-AIMLResearchBootcamp23/NLP_Midterm-Michael-Ari-Jason-Alex-/blob/main/midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import
import gensim
import gensim.downloader as api
import re
import nltk
from nltk.corpus import words
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.spatial.distance import cosine

In [ ]:
#Reads Files
directory = "/content/drive/MyDrive/Classroom/'23 MehtA+ AI ML Research Bootcamp/Group Work/week3/texts"

all_texts = []
all_file_names = []
for filename in os.listdir(directory):
  if '.txt' in filename:
    all_file_names.append(filename)
all_file_names = sorted(all_file_names)

for filename in all_file_names:
  file_path = os.path.join(directory, filename)
  file1 = open(file_path, "r")
  cons_text = file1.read()
  all_texts.append(cons_text)

In [ ]:
#Creates a full dataframe
texts_df = pd.DataFrame(all_texts, columns =['Texts'])
filenames_df = pd.DataFrame(all_file_names, columns = ["FileName"])
full_training_data = pd.concat([filenames_df, texts_df], axis = 1) # Full data with texts & names

Preprocessing

In [ ]:
def preprocessing(doc):
  newdoc = ''
  doc = doc.lower()
  doc = str(doc)
  for word in doc.strip().split():
    for character in word:
      if character in '.,<>!@#$%^&*()/?;:\'\"[{]}=+-_\|:’`':
        no_punc = word.replace(character,'')
        doc = doc.replace(word, no_punc)
        word = no_punc
      elif character not in "abcdefghijklmnopqrstuvwxyz ":
        doc = doc.replace(word,'')
        break
    if(not re.match('[^\d\W]+[a-z][a-z]',word) or len(word) < 3 or 'http' in word):
      doc = doc.replace(word,'')
      continue
    else:
      newdoc = newdoc + " " + word
  return newdoc

In [ ]:
full_training_data["Texts"] = full_training_data["Texts"].apply(preprocessing)

In [ ]:
#Prints out file names with index numbers next to them to reference for later inquiries
for i in range(48):
  print(i, str(full_training_data.loc[i,"FileName"].strip()))

0 Permanent magnets reduce bycatch of benthic sharks in an ocean trap fishery.txt
1 Population Changes of Brown Long-Eared Bats (Plecotus auritus) in Bat Boxes at Thetford Forest.txt
2 Predator-Resembling Aversive Conditioning for Managing Habituated Wildlife.txt
3 Preferences for different flagship types in fundraising for nature  conservation .txt
4 Preventing crop raiding by the Vulnerable common hippopotamus Hippopotamus amphibius in Guinea-Bissau.txt
5 Propane exploders and Electronic Guards were ineffective at reducing deer damage in cornfields.txt
6 Re-assessing the effectiveness of wire-marking to mitigate bird collisions with power lines_ A meta-analysis and guidelines for field studies.txt
7 Reactions of Free-Ranging Black Bears to Capsaicin Spray Repellent .txt
8 Real-time feedback promotes energy conservation in the absence of volunteer selection bias and monetary incentives.txt
9 Reducing Mammalian Predation on Eggs by Using a Conditioned Taste Aversion to Deceive Predator

Doc2Vec

In [ ]:
#Training Doc2Vec Model
dataset = api.load("text8")
data = [d for d in dataset]
def tagged_document(list_of_list_of_words):
   for index, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [index])

#training and building the model
data_training = list(tagged_document(full_training_data["Texts"]))
docvecmodel = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=100)
docvecmodel.build_vocab(data_training)
docvecmodel.train(data_training, total_examples=docvecmodel.corpus_count, epochs = docvecmodel.epochs)

In [ ]:
#Doc2Vec asking for similarity
def ask_docvec():
  num1 = int(input("What document number do you want similar documents for?\n"))
  darray3 = docvecmodel.dv.most_similar(num1)
  print("\n\nDoc",full_training_data.loc[num1,"FileName"].strip(), "Similarities")
  for coord in range(5):
    print(full_training_data.loc[darray3[coord][0],"FileName"].strip())

In [ ]:
ask_docvec()

What document number do you want similar documents for?
34


Doc The value of ‘bat boxes’ for attracting hollow_dependent fauna to farm forestry plantations in southeast Queensland.txt Similarities
The value of bat-boxes in the conservation of Pipistrellus pygmaeus in wetland rice paddies.txt
Someplace_like_home_Experience_habitat_selection_a.txt
The Rules of Attraction.txt
Use of Artificial Roost Structures by Bats at the Indianapolis International Airport.txt
Responses of Little Brown Myotis to Exclusion and to Bat Houses.txt


Bag of Words

In [ ]:
#Bag of Words defined
def bagOfWords(full_training_data):
  vectorizer = CountVectorizer(lowercase = True, stop_words='english', strip_accents='ascii', token_pattern=r'\b[^\d\W]+[a-z][a-z]\b', min_df=0.021, max_df=0.9, ngram_range=(1,1))
  vector = vectorizer.fit_transform(full_training_data["Texts"])
  vector_array = vector.toarray()
  return vector_array

In [ ]:
vector_array = bagOfWords(full_training_data)

In [ ]:
# cosine similarity function (between two documents)
def doc_sim(number1, number2):
  return 1-cosine(vector_array[number1],vector_array[number2])

In [ ]:
# iterating through every document and finding the document that is most similar to it
def doc_comparison():
  doc_1 = int(input("What document number do you want similar documents for?\n"))
  base_num = [0,0,0,0,0]
  index_num = [0,0,0,0,0]
  for doc_2 in range(48):
    if doc_1 == doc_2:
      continue
    accuracy = doc_sim(doc_1, doc_2)
    for top in range(5):
      if accuracy > base_num[top]:
        base_num[top] = accuracy
        index_num[top] = doc_2
        break
  print("\n\nDoc",full_training_data.loc[doc_1,"FileName"].strip(), "Similarities")
  for l in range(5):
    print(" -",l,str(full_training_data.loc[index_num[l],"FileName"].strip()))

In [ ]:
doc_comparison()

What document number do you want similar documents for?44


Doc WEAVER-DISSERTATION-2019.txt Similarities
 - 0 UNDERSTANDING WIND ENERGY IMPACTS ON BATS AND TESTING  REDUCTION STRATEGIES IN SOUTH TEXAS.txt
 - 1 Wildlife Society Bulletin - 2019 - Romano - Evaluation of an acoustic deterrent to reduce bat mortalities at an Illinois.txt
 - 2 Use of Artificial Roost Structures by Bats at the Indianapolis International Airport.txt
 - 3 Utilization of artificial shelters by bats (Chiroptera) in three different  types of forest.txt
 - 4 The interaction of descriptive and injunctive social.txt
